In [6]:
from llama_index.readers.obsidian import ObsidianReader
import openai
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml

ModuleNotFoundError: No module named 'llama_index'

In [3]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Build a simple directory

In [6]:
simple_glossary = SimpleDirectoryReader(
    r"/Users/arthursarazin/Documents/coreandgraphs/graphandvisualize/graph"
).load_data()

# Upload Obsidian vault files

#### Using Obsidian reader from LLama Index

In [7]:
#locate obsidian files that make your graph
filepath = r"/Users/arthursarazin/Documents/coreandgraphs/graphandvisualize/graph"

#load the graph files 
graph_files = ObsidianReader(filepath).load_data()
print(list(graph_files))

[Document(id_='cd904f21-8c2e-4f08-a6e9-f41356012779', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="Quels critères pour dire que des données sont de qualité ?\n- Exactitude : les données sont justes et cohérentes\n- Complétude : le jeu de données est complet \n- Pertinence : les données que l'on a sont elles pertinentes pour répondre à nos besoins (et par extension ceux du client ?)\n\nDes données de bonnes qualités sont essentielles pour avoir des donnes fiables ", mimetype=None, path=None, url=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'), Document(id_='5e82c833-b4f2-4a32-b5f8-187513da377f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {valu

# Using manual code to create specific ontology

In [8]:
# Fonction pour extraire les métadonnées YAML et le titre d'un fichier Markdown
def extract_metadata_and_title(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        # Recherchez la section YAML (entre `---` au début et à la fin)
        if content.startswith("---"):
            try:
                yaml_block = content.split("---", 2)[1]
                metadata = yaml.safe_load(yaml_block)
                return os.path.basename(file_path).replace(".md", ""), metadata
            except Exception as e:
                print(f"Erreur lors de l'extraction des métadonnées de {file_path}: {e}")
                return None, None
        return os.path.basename(file_path).replace(".md", ""), None

# Parcourir les fichiers Obsidian et créer des entités et relations
def process_obsidian_notes(directory):
    entities = []
    relations = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                title, metadata = extract_metadata_and_title(file_path)
                if title:
                    # Ajouter l'entité basée sur le titre
                    entities.append({"name": title})
                    
                    # Ajouter les relations basées sur les propriétés YAML
                    if metadata:
                        for key, value in metadata.items():
                            relations.append({
                                "type": key,  # Nom de la propriété comme relation
                                "source": title,  # Titre du fichier comme entité source
                                "target": value,  # Valeur de la propriété comme entité cible
                            })
    return entities, relations



In [9]:
# Exemple d'utilisation
obsidian_dir = r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\graph"
entities, relations = process_obsidian_notes(obsidian_dir)

print(relations)

[]


# Set LLM (OpenAI)

In [11]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings ()

In [12]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [13]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************mY4A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [50]:
# Instantiate vector store
simple_glossary = SimpleDirectoryReader(
    r"/Users/arthursarazin/Documents/coreandgraphs/graphandvisualize/graph"
)

# Instantiate graph store 

In [51]:
graph_store = SimpleGraphStore()
graph_storage_context = StorageContext.from_defaults(graph_store=graph_store)

# Instantiate ontology store

In [52]:
onto_store = SimpleGraphStore()
onto_storage_context = StorageContext.from_defaults(graph_store=onto_store)

# Construct Index

## Construct Vector Store Index

In [ ]:
vector_index = VectorStoreIndex.from_documents(
    documents = simple_glossary.load_data(),
    show_progress=True)

Parsing nodes:   0%|          | 0/28 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/28 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
# save index to disk
vector_index.set_index_id("vector_index")
vector_index.storage_context.persist("vector")

## Construct knowledge graph index 

In [55]:
def process_document(doc):
    # This function will be called for each document
    print(f"Processing document: {doc.doc_id}")
    return doc

In [56]:
# Assuming you have your graph files loaded
print(f"Starting to process {len(graph_files)} documents...")

Starting to process 28 documents...


In [22]:
# Wrap your document processing in a progress bar
graph_index = KnowledgeGraphIndex.from_documents(
        documents=[process_document(doc) for doc in graph_files],  # process one document at a time
        max_triplets_per_chunk=2,
        storage_context=graph_storage_context,
        include_embeddings=True,
    show_progress=True,
)


Processing document: e65fc8f3-5550-449e-8903-ff85ff12dd68
Processing document: d981110c-e323-401e-9c1d-4f007538806a
Processing document: a2c281da-35c4-4ea4-b20c-4f4ab740f6fe
Processing document: 40083469-d412-4abf-9f34-ad22b59feeb2
Processing document: e9171569-8b94-4dfe-a63a-6ee24a9a52b6
Processing document: 5a074c14-868c-48c9-bc6b-edf9b7ced92f
Processing document: 16429528-5bef-41f2-bb06-46f5350a4e11
Processing document: 2c8d574d-cec5-4855-b2fd-76e18301a5a4
Processing document: 24680154-1453-4951-82fb-dd8c65776680
Processing document: 0eb7e13b-048f-4129-a7bf-a5b937a81bd5
Processing document: b678f851-0f20-4379-a56b-a56f46d51b65
Processing document: fecf21f7-31ed-44dd-9efe-ea5e89a33f9a
Processing document: 5e901a69-450c-4dd6-9cf5-801efdffe81a
Processing document: 197955c5-1979-40ef-b025-50939018ba1a
Processing document: 165985f4-891d-481e-861d-f7ec74013fe1
Processing document: 6d28933b-2dc9-4b38-9cc8-d5200c552394
Processing document: 7f479604-7ece-4896-81da-1808d58b2879
Processing doc

Parsing nodes:   0%|          | 0/28 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
print(graph_index)

In [24]:
graph_index.set_index_id("graph_index")
graph_index.storage_context.persist("knowledge_graph")
graph_storage_context.persist(persist_dir="knowledge_graph")


## Construct knowledge with ontology as assistant

In [25]:
onto_store = SimpleGraphStore()
onto_storage_context = StorageContext.from_defaults(graph_store=onto_store)

In [26]:
# Configurer DynamicLLMPathExtractor avec les entités et relations extraites
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=None,  # Si vous ne limitez pas les types d'entités
    allowed_relation_types=None,  # Si vous ne limitez pas les types de relations
    allowed_entity_props=None,
    allowed_relation_props=None,
)

In [27]:
# Résumé des entités et relations extraites
print("Entités :")
for entity in entities:
    print(entity)

print("\nRelations :")
for relation in relations:
    print(relation)

Entités :

Relations :


In [28]:
onto_index = PropertyGraphIndex.from_documents(
    documents=[process_document(doc) for doc in graph_files],
    llm=llm,
    storage_context=onto_storage_context,
    embed_kg_nodes=True,
    kg_extractors=[kg_extractor],
    show_progress=True,
    graph_store=onto_store 

)

Processing document: d30d26d2-8eeb-4600-b3b4-e37a1a46fa2b
Processing document: 6e3e2815-a9f9-495d-b4ab-789ada6782f0
Processing document: 5c1f06c1-fba9-4105-a2e4-1bc0e37ade9c
Processing document: 04e1dafb-08e6-4832-98c9-b89210c6f3ce
Processing document: c73595f5-685b-4006-ba57-ef54cd0eb938
Processing document: 974297b0-f4d7-4039-a029-cf5cff84d506
Processing document: ff49248c-0969-47fd-ae45-e8181378acfa
Processing document: cf06c6e9-f01d-4fe4-910e-e2f39764a50a
Processing document: 5e18d055-afdd-4746-8585-5c14dce5cf60
Processing document: 86049378-8247-4fd7-96ee-3cbfe578f7ac
Processing document: c7565f1d-080a-4c83-a369-ecc6cbe44c7a
Processing document: 189f05e0-1fe3-47bb-9971-33e08c0092e9
Processing document: e554b0f3-13e5-4853-95de-23bc3256ee35
Processing document: a8a1a003-9060-4313-b16d-0d7a56ef24d7
Processing document: 4ccf3d62-7ffb-4660-be65-e78d06822b84
Processing document: 13710c8f-96c1-459f-bc0c-271736f6606f
Processing document: 3625a7fc-165c-4ee3-aa70-06ace2867a65
Processing doc

Parsing nodes:   0%|          | 0/28 [00:00<?, ?it/s]

Extracting and inferring knowledge graph from text:   0%|          | 0/28 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Extracting and inferring knowledge graph from text:   4%|▎         | 1/28 [00:01<00:46,  1.72s/it]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 38/38 [00:37<00:00,  1.00it/s]


In [29]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x2c2080b10>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x2c1195390>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={'Exactitude': [0.015441572293639183, 0.003143112640827894, 0.04897480458021164, -0.023916300386190414, 0.014095205813646317, 0.06652942299842834, 0.053384508937597275, 0.022607628256082535, -0.038246773183345795, -0.024076776579022408, -0.007392427418380976, 0.03986386954784393, -0.07254062592983246, 0.023015538230538368, -0.03190811723470688, 0.0692232996225357, 0.05948972329497337, -0.027217192575335503, 0.0029663408640772104, -0.008480524644255638, 0.008642234839498997, -0.003019848605617881, -0.02687036246061325, -0.018865369260311127, -0.002895112382248044, 0.015625955536961555, -0.01422868762165308, -0.010296400636434555, -0.0

In [30]:
# save index to disk
onto_index.set_index_id("onto_index")
onto_index.storage_context.persist("onto_graph")
onto_storage_context.persist(persist_dir="onto_graph")